# Dask Match Demonstration

We make a Dask client, mostly so we can use the dashboard.

In [ ]:
from dask.distributed import Client
client = Client()

## Example DataFrame

In [ ]:
import dask_expr as dx

df = dx.datasets.timeseries(
    start="2000-01-01", 
    end="2000-12-30", 
    freq="100ms",
)
df

## Expression trees and optimization

In [ ]:
%%time
out = df[df.id == 1000].x.sum()
out.compute()

In [ ]:
out.pprint()

In [ ]:
out.optimize(fuse=False).pprint()

In [ ]:
out.optimize().pprint()

In [ ]:
%%time
out.optimize().compute()

## Class structure

In [ ]:
out = (df.x + 1)
out

Dataframe/Series/Index objects live in `collections.py`

In [ ]:
type(out)

Collections used to hold `_meta`, `divisions`, `_name`, and `__dask_graph__`.  Now they hold just `expr`, which computes these things based on user inputs.

In [ ]:
out.__dict__

In [ ]:
out.expr

Expressions have a type hierarchy which reflects user commands

In [ ]:
type(out.expr)

In [ ]:
type(out.expr).mro()

Expressions are composed of a *type* or *Operation* (like `Add`) and *operands*, (like `left` and `right`)

In [ ]:
out._parameters

In [ ]:
out.operands

In [ ]:
type(out.left)

In [ ]:
dict(
    zip(
        out.left._parameters, 
        out.left.operands,
    )
)

In [ ]:
dict(
    zip(
        out.left.frame._parameters, 
        out.left.frame.operands,
    )
)